<a href="https://colab.research.google.com/github/bhadreshpsavani/UnderstandingNLP/blob/master/UnifiedQAExperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 1.3MB 5.5MB/s 
     |████████████████████████████████| 890kB 29.6MB/s 
     |████████████████████████████████| 2.9MB 49.2MB/s 
     |████████████████████████████████| 1.1MB 49.7MB/s 


In [2]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [2]:
# from torch import cuda
# device = 'cuda' if cuda.is_available() else 'cpu'
# print(device)

cuda


## Import Data

In [6]:
import json
import pandas as pd
def squad2_json_to_df(file):
    f = open (file, "r") 
    data = json.loads(f.read())               #loading the json file.
    iid = []                                  
    tit = []                                  #Creating empty lists to store values.
    con = []
    Que = []
    Ans_st = []
    Txt = []
    
    for i in range(len(data['data'])):       #Root tag of the json file contains 'title' tag & 'paragraphs' list.
        
        title = data['data'][i]['title']
        for p in range(len(data['data'][i]['paragraphs'])):  # 'paragraphs' list contains 'context' tag & 'qas' list.
            
            context = data['data'][i]['paragraphs'][p]['context']
            for q in range(len(data['data'][i]['paragraphs'][p]['qas'])):  # 'qas' list contains 'question', 'Id' tag & 'answers' list.
                
                question = data['data'][i]['paragraphs'][p]['qas'][q]['question']
                Id = data['data'][i]['paragraphs'][p]['qas'][q]['id']
                for a in range(len(data['data'][i]['paragraphs'][p]['qas'][q]['answers'])): # 'answers' list contains 'ans_start', 'text' tags. 
                    
                    ans_start = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['answer_start']
                    text = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['text']
                    
                    tit.append(title)
                    con.append(context)
                    Que.append(question)                    # Appending values to lists
                    iid.append(Id)
                    Ans_st.append(ans_start)
                    Txt.append(text)

    print('Done')      # for indication perpose.
    new_df = pd.DataFrame(columns=['Id','title','context','question','ans_start','text']) # Creating empty DataFrame.
    new_df.Id = iid
    new_df.title = tit           #intializing list values to the DataFrame.
    new_df.context = con
    new_df.question = Que
    new_df.ans_start = Ans_st
    new_df.text = Txt
    print('Done')      # for indication perpose.
    final_df = new_df.drop_duplicates(keep='first')  # Dropping duplicate rows from the create Dataframe.
    return final_df

def squad1_json_to_df(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
#     ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
#     js['q_idx'] = ndx
    main = m[['id','question','context','answers']].set_index('id').reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [7]:
dev2_df = squad2_json_to_df('dev-v2.json')

FileNotFoundError: ignored

In [ ]:
dev2_df.head()

In [ ]:
dev2_df.columns

In [ ]:
def input_formatter(row):
  paragraph=''
  question=row['question']
  paragraph+=row['title']
  paragraph += row['context'].strip().replace("\n", "").replace("\t", "")
  paragraph = paragraph.replace("\t", "").replace("   ", " ").replace("  ", " ").replace("\n", " ")
  question = question.replace("\t", "").replace("   ", " ").replace("  ", " ").replace("\n", " ")
  input_qt = question.strip() + ' \\n '+ paragraph.strip()
  return input_qt

In [8]:
from tqdm import tqdm
tqdm.pandas()

## Import Model:

In [11]:
model_name="allenai/unifiedqa-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model =  T5ForConditionalGeneration.from_pretrained(model_name)

In [5]:
input = "Which is best conductor? \\n (A) iron (B) feather (C) wood (D) plastic"
input_ids = tokenizer.encode(input, padding='max_length', max_length=384, truncation=True, return_tensors='pt')
attention_mask = input_ids!=0
attention_mask = attention_mask.long()

In [6]:
dummy_output = model(input_ids, attention_mask)
print(dummy_output)

ValueError: ignored

In [ ]:
# Export to ONNX format
torch.onnx.export(
    model,
    args=(sample_ids, sample_mask),
    f='./distilbert_cortex_week10_cause_detection.onnx',
    input_names=['input_ids', 'attention_mask'],
    output_names=['output']
)

In [ ]:
import onnxruntime as ort
import numpy as np

In [ ]:
sample_ids = (sample_data["ids"].reshape(1, -1)).numpy()
sample_mask = (sample_data["mask"].reshape(1, -1)).numpy()

In [ ]:
outputs = ort_session.run(None, {
    "input_ids": sample_ids,
    "attention_mask": sample_mask
})

print(outputs[0])

In [9]:
import torch

In [12]:
class Pytorch_to_Torchscript(torch.nn.Module):
  def __init__(self):
    super(Pytorch_to_Torchscript, self).__init__()
    self.model = T5ForConditionalGeneration.from_pretrained("allenai/unifiedqa-t5-small").cuda()
  def forward(self, data, attention_mask=None):
    return self.model(data.cuda(), attention_mask,cuda())

In [8]:
PATH = "unifiedqa_small.pt"
torch.save(model, PATH)

In [34]:
# !git clone https://huggingface.co/allenai/unifiedqa-t5-base

Cloning into 'unifiedqa-t5-base'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [ ]:
def run_model(input_string, **generator_args):
  input_dict = tokenizer(input_string, max_length=512,  truncation=True, return_tensors="pt")
  input_ids = input_dict['input_ids'].to(device)
  attention_mask = input_dict['attention_mask'].to(device)
  res = model.generate(input_ids=input_ids, attention_mask=attention_mask, **generator_args)
  res = res.to('cpu')
  return [tokenizer.decode(x) for x in res]

In [ ]:
run_model("Which is best conductor? \\n (A) iron (B) feather (C) wood (D) plastic",
         temperature=0.9, num_return_sequences=4, num_beams=20)

['iron',
 'iron (B) iron (C) feather (D) wood (D) plastic',
 'iron (B) iron (C) feather (D) wood',
 'iron (B) iron (C) feather (C) wood (D) plastic']

In [ ]:
run_model("Scott filled a tray with juice and put it in a freezer. The next day, Scott opened the freezer. How did the juice most likely change? \\n (A) It condensed. (B) It evaporated. (C) It became a gas. (D) It became a solid.")

['It became a solid.']

In [ ]:
run_model("Which is best conductor? \\n (A) iron (B) feather")

['iron']

## Inference:

In [ ]:
dev2_df['inputs']=dev2_df.apply(input_formatter, axis=1)

In [ ]:
dev2_df['inputs'][:10]

0     In what country is Normandy located? \n Norman...
4     When were the Normans in Normandy? \n NormansT...
5     When were the Normans in Normandy? \n NormansT...
8     From which countries did the Norse originate? ...
12    Who was the Norse leader? \n NormansThe Norman...
16    What century did the Normans first gain their ...
17    What century did the Normans first gain their ...
18    What century did the Normans first gain their ...
20    Who was the duke in the battle of Hastings? \n...
23    Who ruled the duchy of Normandy \n NormansThe ...
Name: inputs, dtype: object

In [ ]:
dev2_df['prediction'] = dev2_df['inputs'].progress_apply(run_model)

100%|██████████| 10388/10388 [23:10<00:00,  7.47it/s]


In [ ]:
"""Evaluation utilities."""
import argparse
import collections
import json
import os
import re
import string
import sys
import numpy as np

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([("exact", 100.0 * sum(exact_scores.values / total)), ("f1", 100.0 * sum(f1_scores.values / total)),("total", total)])

In [ ]:
dev2_df['text'] = dev2_df['text'].progress_apply(lambda answer: tokenizer.decode(tokenizer.encode(answer)))

100%|██████████| 10388/10388 [00:03<00:00, 2717.12it/s]


In [ ]:
dev2_df['exact_match'] = dev2_df.apply(lambda row:compute_exact(row['text'], row['prediction'][0]), axis=1)
dev2_df['f1_score'] = dev2_df.apply(lambda row:compute_f1(row['text'], row['prediction'][0]), axis=1)
make_eval_dict(dev2_df['exact_match'], dev2_df['f1_score'])

OrderedDict([('exact', 52.25259915286172),
             ('f1', 72.89594377792447),
             ('total', 10388)])

In [ ]:
dev2_df.query('f1_score==0')[['prediction', 'text','f1_score']].head(10)

,prediction,text,f1_score
26,[Christian],Catholic,0.0
27,[Christian],Catholic orthodoxy,0.0
29,[nortmann],Viking,0.0
30,[nortmann],"Norseman, Viking",0.0
41,[the river Epte],Seine,0.0
44,[ ⁇ no answer>],Rollo,0.0
47,[norse],Catholicism,0.0
71,[Raimbaud],Oursel,0.0
74,[the Armenian state],Turkish forces,0.0
77,[italo-Norman],Norman mercenary,0.0


### Converting Model to ONNX:

In [8]:
import os

In [7]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 53180 (delta 44), reused 66 (delta 27), pack-reused 53081
Receiving objects: 100% (53180/53180), 40.00 MiB | 13.43 MiB/s, done.
Resolving deltas: 100% (37015/37015), done.


In [9]:
os.chdir('/content/transformers/src/transformers')

In [10]:
os.getcwd()

'/content/transformers/src/transformers'

In [ ]:
model = 

In [14]:
!python convert_graph_to_onnx.py --framework pt --model allenai/unifiedqa-t5-small unified_qa_small.onnx

2020-11-25 13:23:01.142167: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1

====== Converting model to ONNX ======
ONNX opset version set to: 11
Loading pipeline (model: allenai/unifiedqa-t5-small, tokenizer: allenai/unifiedqa-t5-small)
Some weights of the model checkpoint at allenai/unifiedqa-t5-small were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Error while converting the model: Folder /content/transformers/src/transformers is not empty, aborting conversion
